Experiment 1 with resnet18 transfer learning
============================================

In [ ]:
from __future__ import print_function

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import csv
import gc
import torchnet as tnt
from utils import *
from classes import *
from tqdm import tqdm_notebook # for-loop progress bar in notebook

# plt setup and the gpu setup
plt.ion()
use_gpu = torch.cuda.is_available()


In [ ]:
# Create custom Dataset object
# A unique object will need to be created for each book.
# It will take in all the transforms = a dict with key, val where:
#   key=phase, val=tranform to use

# To create custom Dataset
from torch.utils.data.dataset import Dataset
from torchvision.datasets.folder import *

class BookDatasetContainer:
    __samples = None
    __images = []
    
    def __init__(self, root, loader=default_loader):  
        # extension
        IMG_EXTENSIONS = ['.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif']
        
        # Read in all images
        classes, class_to_idx = find_classes(root)
        # If on windows, use the 2 argument versin of make_dataset
        if os.name == 'nt':
            samples = make_dataset(root, class_to_idx)
        else:
            samples = make_dataset(root, class_to_idx, IMG_EXTENSIONS)
        self.__samples = samples
        self.__images = [(loader(path), target) for path, target in samples]
            
    def __len__(self):
        return len(self.__samples)
    
    def __getitem__(self, index):
        return self.__images[index]
    
    # Inner BookDataset class uses the BookDatasetContainer's images, adding transform functionality
    class BookDataset(Dataset):
        __dataset_container = None
        __transform = None

        def __init__(self, dataset_container, transform):
            self.__dataset_container = dataset_container
            self.__transform = transform

        def __getitem__(self, index):
            sample, target = self.__dataset_container[index]
            if (self.__transform is not None):
                sample = self.__transform(sample)
            return sample, target

        def __len__(self):
            return len(self.__dataset_container)
    
    # Method of BookDatasetContainer to create a BookDataset with a specific transform
    def create_dataset(self, transform):
        return BookDatasetContainer.BookDataset(self, transform)

In [ ]:
# ========================================
# Step 1. define data transform
#
# Including different forms of data augmentation
# One will include nearly all types (excluding random crops, etc. that may remove handwriting.)
# The other will include a selected set of augmentations
# Keeping 'train', 'val', and 'test' transforms just in case we want to include different functionalities
# ========================================

# Need the __name__ check to make multiprocessing work on Windows for some reason
if __name__ == '__main__':
    print("Set up data transforms.")
    img_input_size = 56

    selected_transforms = {
        'train': transforms.Compose([
            transforms.RandomHorizontalFlip(),
            # RandomRotation does not seem to be working on Windows right now
    #         transforms.RandomRotation(45),

            transforms.ColorJitter(brightness=0, contrast=0, saturation=0, hue=0.5),
            transforms.RandomGrayscale(p=0.1),

            transforms.Grayscale(), # not sure why the current input is not grayscale, do grayscale conversion
            transforms.Resize((img_input_size,img_input_size)),
            transforms.ToTensor(),
        ]),
        # should not do random transformation in val or test set
        'val': transforms.Compose([
            transforms.Grayscale(),
            transforms.Resize((img_input_size,img_input_size)),
            transforms.ToTensor(),
        ]),
        'test': transforms.Compose([
            transforms.Grayscale(),
            transforms.Resize((img_input_size,img_input_size)),
            transforms.ToTensor(),
        ]),
    }
    data_transforms = selected_transforms


In [ ]:
# ========================================
# Step 2. define and load data
#
# TODO: Suggest not using data loader to load the images. Because it is slow. Every epochs, you load the 
# image data again to the RAM and then from RAM to GPU RAM. That takes a lot of time. Since the images are 
# <1GB, you can preload the images into RAM first, that will make you training way faster. 

# Note: you only has about 200MB of images, it's small. It should not take much time to load!
# ========================================

# This flag is on if the custom dataset BookDataset should be used
use_custom_dataset = True

# Need the __name__ check to make multiprocessing work on Windows for some reason
# Unfortunately, still getting broken pipe error on Windows with custom dataset.
if __name__ == '__main__':
    print("Create dataset and dataloader")

    # hyperparameter
    # batch_size = 50 # larger batch size is better so you can load more data into gpu and train faster
    batch_size = 50 # larger batch size is better so you can load more data into gpu and train faster

    # data location
#     book_data_dir = "/home/kcho/clab_data/"
    book_data_dir = "C:\\Users\\rahul\\Documents\\work\\BuildUCLA\\data\\printed_with_ids_harsh_filter\\books-preprocessed-images"
    set_types = ['train', 'val', 'test']

    # test books are currently arbitrarily set
    test_books = set(["Albin", "Dryden"])

    # Get the list of all books in the data set
    books_in_data = set([b for b in os.listdir(book_data_dir)
                     if os.path.isdir(os.path.join(book_data_dir, b))])
    
    if use_custom_dataset:
        print('Using custom dataset.')
        book_data_sets = {b : BookDatasetContainer(os.path.join(book_data_dir, b))
                          for b in books_in_data}
        
        book_data_loaders = {b : {t : torch.utils.data.DataLoader(
                                  book_data_sets[b].create_dataset(data_transforms[t]),
            batch_size=batch_size,
            shuffle=True, # make sure you shuffle the data
#             num_workers=0)
          num_workers=4)
                                  for t in set_types}
                             for b in books_in_data}
        
    else:
        print('Using default ImageFolder dataset.')
        # Create a dict of datasets for each book
        book_data_sets = {b : {t : datasets.ImageFolder(os.path.join(book_data_dir, b), 
                                                        transform = data_transforms[t])#, transform=test_transform)
                              for t in set_types}
                         for b in books_in_data}


        # create a dict of dataloaders, book_data_loaders['Albin']['train']
        book_data_loaders = {b : {t : torch.utils.data.DataLoader(book_data_sets[b][t],
                                                                  batch_size=batch_size,
                                                                  shuffle=True, # make sure you shuffle the data
                                                                  num_workers=4)
                                  for t in set_types}
                             for b in books_in_data}
    

    print("data loading complete")



In [ ]:
# ========================================
# Check your data to make sure it is correct
# ========================================

# Need the __name__ check to make multiprocessing work on Windows for some reason
if __name__ == '__main__':
    for data in book_data_loaders['Defoe']['train']:
        # get the inputs
        inputs, labels = data

        # plot to verify the input is correct
        input1 = inputs[0].numpy()
        print(input1.shape)
        input1 = np.swapaxes(input1,0,2).squeeze()
        print(input1.shape)
        plt.imshow(input1,cmap='gray')

        break

In [ ]:
# ========================================
# Step 3. define model structure
#
# TODO: Stick with MNIST for now because it is smaller network and faster to train. Note that you lose
# a lot of inofmration when yo udownsample the image to 28x28. THat's okay, you optimize other parts first, e.g.,
# data loading before you optimize your model.
# ========================================
from lib.playground.utee import selector
from lib.playground.mnist import model
import os

def create_model_architecture(model_type='mnist'):
    """
    params model_type: the type of model, for now, support mnist and resnet18    
    """
    if model_type == 'mnist':
        print('using pretrained mnist model')
        
        # load the model from the playground library
        model_annotation, ds_fetcher, is_imagenet = selector.select('mnist')
        
        # remove last layer
        removed = list(model_annotation.model.children())[:-1]
        
        # add a front layer to account for new input
        # IMPORTANT, we need to update the self.input_dims of the MLP class
        removed = [nn.Linear(img_input_size*img_input_size,28*28), nn.ReLU()] + removed
        
        # formulate the layers
        model_annotation.model=torch.nn.Sequential(*removed)
        
        # add the new fc layer
        model_annotation.model.fc = torch.nn.Linear(256,2).cuda()
        
        # update the self.input_dims of the network
        model_annotation.input_dims = img_input_size * img_input_size                

    elif model_type == 'resnet18':    
        print("Transferring resnet18 and retraining with annotations dataset.")    
        model_annotation = models.resnet18(pretrained=True)
        num_params = sum(1 for i in model_annotation.parameters())

        # There are 10 layers (model_ft.children()) in resnet18
        # Freezing the first half of resnet18, freezing all params for layers 1-5
        max_layer = 5
        curr_layer = 1
        last_layer = None
        for child in model_annotation.children():
            if curr_layer <= max_layer:
                for param in child.parameters():
                    param.requires_grad = False
                last_layer = child
                curr_layer = curr_layer + 1
            else:
                break

        # Replace the final fully connected layer to perform binary classification
        num_ftrs = model_annotation.fc.in_features
        model_annotation.fc = nn.Linear(num_ftrs, 2)
        

    # return
    if use_gpu:
        return model_annotation.cuda()
    else:
        return model_annotation.cpu()

    
net = create_model_architecture(model_type='mnist')
print(net)

In [ ]:
# ========================================
# Step 4. define the training process.
#
# TODO: The basic process of train and validation is defined. Please implement the overall average validation 
# confusion matrix, meaning for each validation (after running all epochs), get the confusion matrix for that book,
# repeat this for the rest of 10 books. THen get the overall performance. Also, implement the early-stop as you
# originally has in your code. :) I removed them here for clarity. You can add them back. 
# ========================================

def train(model, criterion, optimizer, data_loaders, num_epochs=25, early_stopping = None):
    since = time.time()

    # stop the training, validation, and test loop
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        # looping parameters
        running_loss = 0.0
        confusion_matrix = tnt.meter.ConfusionMeter(2)
        
        # loop through train and val phase in each epoch
        for phase in ['train', 'val']:
            # check train or val
            if phase == 'train':                
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            # Iterate over each book
            running_loss = 0.0
            confusion_matrix = tnt.meter.ConfusionMeter(2)
#             for book in tqdm_notebook(data_loaders[phase]): 
            for book in data_loaders[phase]:
                for data in data_loaders[phase][book]:
                    # get the inputs;  wrap them in Variable and make them into gpu or not
                    inputs, labels = data # input size: [5, 1, 28, 28] ; keep the dummy color channel:1
                    if use_gpu:
                        inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
                    else:
                        inputs, labels = Variable(inputs), Variable(labels)
                    
                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    outputs = model(inputs)
                    _, preds = torch.max(outputs.data, 1)
                    loss = criterion(outputs, labels)
                    
                    # back
                    if phase == 'train': 
                        loss.backward()
                        optimizer.step()

                    # Add to confusion matrix
                    confusion_matrix.add(outputs.data, labels.data)

                    # statistics
                    running_loss += loss.data[0] * inputs.size(0) 
            
            # report evaluation
            print('Phase:%s' %phase)
            print('Confusion matrix:\n', confusion_matrix.conf)
            print('validation loss', running_loss)
    
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    
    return model

In [ ]:
# ========================================
# Step 5. execute the train process
# ========================================

# get the model
model = create_model_architecture()

# train parameters
# num_training_epochs = 5
num_training_epochs = 1

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
earlyStoppingCriteria = EarlyStopping(min_delta = 1e-4, patience=5)

# dataloader parameters
cross_val_loaders = {}

# leave-one-book-out cross validation
for val_book in books_in_data:
    
    # define the train and validation loaders
    train_books = books_in_data - set([val_book])    
    cross_val_loaders["train"] = {b : book_data_loaders[b]["train"] for b in train_books}
    cross_val_loaders["val"] = {b : book_data_loaders[b]["val"] for b in [val_book]}

    # train
    trained_model = train(model,
                          criterion,
                          optimizer,
                          cross_val_loaders,
                          num_epochs=num_training_epochs)
    
    # remove the break as needed
    break

print("training complete")

In [ ]:
# os.path.join('', 'mnist.pth')